# Overview

This notebook shows how to extract the relevant dataset (target position, GVD, TOD, number of accelerated ions between 5 and 20 MeV) from the raw BELLA data, stored at NERSC under `/global/cfs/cdirs/m3239/ip2data/bella_pw_data/`.

This notebook should be run at NERSC, in order to have access to the relevant data.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tqdm
import re

# Functions to extract data

In [ ]:
def extract_df( scan_number ):
    """
    Extract the dataset (target position, GVD, TOD, number of accelerated ions) from the raw BELLA data, for a given scan.

    Parameters
    ----------
    scan_number : int
        The identification number of the scan to extract.

    Returns
    -------
    pd.DataFrame
        The dataset extracted from the raw data, with one row per shot of the scan.
    """

    # Open s file: records input parameters for each shot of the scan
    date = "03-Mar/24_0328"
    path = "/global/cfs/cdirs/m3239/ip2data/bella_pw_data/PW/Y2024"
    s_file_location = f"{path}/{date}/analysis/s{scan_number}.txt"
    s_file = pd.read_csv(s_file_location, sep='\t')

    # Extract z position
    laser_focus_position = 12 # mm
    if 'IP2-TC-ESP5 Position.Axis 2 Alias:Preplasma Z (MFA)' in s_file:
        z_target = s_file['IP2-TC-ESP5 Position.Axis 2 Alias:Preplasma Z (MFA)']
    else:
        print('Scan %d: z location not found ; reading it in the ECS Live dumps' %scan_number)
        file_path = f"/global/cfs/cdirs/m3239/ip2data/bella_pw_data/PW/Y2024/{date}/ECS Live dumps/Scan{scan_number}.txt"
        device_name = 'IP2-TC-ESP5'
        property_name = 'Position.Axis 2'
        z0 = extract_device_property(file_path, device_name, property_name)
        print('Found: %.3f' %z0)
        z_target = z0 + 0*s_file['DDG-AA-ShotCntrl shot #'] # make array of the same length
    # Motors are in mm. Here we convert to microns.
    z_target = (z_target - laser_focus_position)*1e3

    # Extract TOD (third-order dispersion)
    if 'DAZZLER-LT-3rdorder value1' in s_file:
        tod = s_file['DAZZLER-LT-3rdorder value1'] # in fs^3
    else: # if not stored, it is 0 by default
        print('Scan %d: TOD not found ; assuming 0' %scan_number)
        tod = 0*s_file['DDG-AA-ShotCntrl shot #'] # make array of the same length

    # Extract GVD
    gvd = s_file['STAGE-1BL-Compression Position.Axis 1 Alias:Compression']

    # Extract total proton number for each shot in the scan
    shot_numbers = s_file['DDG-AA-ShotCntrl shot #'].astype('int')
    n_protons = np.zeros( len(shot_numbers) )
    for i, shot_number in tqdm.tqdm( enumerate(shot_numbers) ):
        spectrum_file = f"{path}/{date}/analysis/superfacility/Scan{str(scan_number).zfill(3)}/24_0328_Scan{str(scan_number).zfill(3)}_{str(shot_number).zfill(3)}.txt"
        n_protons[i] = total_proton_number( spectrum_file )

    return pd.DataFrame( {'z_target_um': z_target,
                          'TOD_fs3': tod,
                          'GVD': gvd,
                          'n_protons': n_protons} )

In [ ]:
def total_proton_number( spectrum_file, E_min=5, E_max=20 ):
    """
    Compute the total number of protons between E_min and E_max,
    from a data file containing the proton spectrum.

    Parameters
    ----------
    spectrum_file : str
        Path to the spectrum file.
    E_min : float, optional
        Minimum energy, in MeV.
    E_max : float, optional
        Maximum energy, in MeV.

    Returns
    -------
    float
        Total number of protons between E_min and E_max in 1/sr.
    """
    # Read the spectrum file
    spectrum = pd.read_csv(spectrum_file, delim_whitespace=True, skiprows=1,
                           names=['#energy (MeV)', 'number (1/MeV/sr)', 'Grayvalue', 'TP spectrometer'])

    # Compute energy steps
    dE = np.diff(spectrum['#energy (MeV)'].to_numpy())
    dE = np.concatenate( (dE, np.array([0])) ) # make it the same length as other arrays
    spectrum['dE (MeV)'] = dE

    # Integrate the spectrum between E_min and E_max
    selection = (spectrum['#energy (MeV)'] > E_min) & (spectrum['#energy (MeV)'] < E_max)
    n_tot = ( spectrum['dE (MeV)'] * spectrum['number (1/MeV/sr)'] )[selection].sum()

    return n_tot

In [ ]:
def extract_device_property(file_path, device_name, property_name):
    """
    Low-level function to extract data from the ECS Live dumps.
    """
    with open(file_path, 'r') as file:
        content = file.read()

    # Regular expression to match the device section
    device_pattern = re.compile(rf'\[Device \d+\]\nDevice Name = "{device_name}"(.*?)\n\n', re.DOTALL)
    device_match = device_pattern.search(content)

    if device_match:
        device_content = device_match.group(1)

        # Regular expression to match the specific property
        property_pattern = re.compile(rf'{property_name} = "(.*?)"')
        property_match = property_pattern.search(device_content)

        if property_match:
            return float(property_match.group(1))
        else:
            return f"Property {property_name} not found in device {device_name}."
    else:
        return f"Device {device_name} not found in the file."

# Data extraction

In [ ]:
# Extract all available scans
df_44 = extract_df(44) # GVD scan
df_51 = extract_df(51) # TOD scan
df_61 = extract_df(61) # Z scan
df_62 = extract_df(62) # stability scan at z = 100 um
df_63 = extract_df(63) # stability scan at z = 50 um

In [ ]:
# Merge all data into one dataset
df = pd.concat( [ df_44, df_51, df_61, df_62, df_63 ] )

In [ ]:
# Save as csv file
df.to_csv('experimental_data.csv')

# Data visualization

In [ ]:
plt.clf()
ax = plt.figure().add_subplot(projection='3d')

ax.scatter( df['TOD_fs3'], df['z_target_um'], df['n_protons'], c=df['n_protons'])
ax.view_init(elev=40., azim=40, roll=0)
plt.xlabel('TOD_fs3')
plt.ylabel('z_target_um')

In [ ]:
plt.plot( df_44['GVD'], df_44['n_protons'], 'o')
plt.title('GVD scan')
plt.xlabel('GVD')
plt.ylabel('Number of protons (1/sr)')

In [ ]:
plt.plot( df_61['z_target_um'], df_61['n_protons'], 'o')
plt.title('Z scan')
plt.xlabel('z_target_um')
plt.ylabel('Number of protons (1/sr)')

In [ ]:
plt.plot( df_51['TOD_fs3'], df_51['n_protons'], 'o')
plt.title('TOD scan')
plt.xlabel('TOD_fs3')
plt.ylabel('Number of protons (1/sr)')